In [1]:
%cd /work/adapting-CLIP-VGPs/
import clip

GPU = 7
model, _ = clip.load('ViT-L/14', device=GPU)

/work/adapting-CLIP-VGPs


In [2]:
model

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)


In [3]:
from utils.vgp_data import FlickrVGPsDataset
from torch.utils.data import DataLoader

test_dataset = FlickrVGPsDataset(data_type='test')

In [4]:
demo_phrases = test_dataset[0]['phrases']
demo_phrases

['Several climbers', 'Seven climbers']

In [5]:
phrases_tensor = clip.tokenize(demo_phrases).to(GPU)
phrases_tensor

tensor([[49406,  5560, 47648, 49407,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [49406,  5757, 47648, 49407,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0, 

In [6]:
text_ft = model.encode_text(phrases_tensor)
text_ft

tensor([[-0.2708,  0.2357, -0.0625,  ...,  0.1311, -0.4551, -0.2832],
        [-0.5215,  0.1737, -0.4531,  ...,  0.1414, -0.0684, -0.7246]],
       device='cuda:7', dtype=torch.float16, grad_fn=<MmBackward>)

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(text_ft.cpu().detach())[0,1]

0.8868458468103846

In [8]:
def text_cos_sim(phrases):
    phrases_tensor = clip.tokenize(phrases).to(GPU)
    text_ft = model.encode_text(phrases_tensor)
    return cosine_similarity(text_ft.cpu().detach())[0,1]

In [9]:
# from tqdm import tqdm

# thres = 0.7
# phrase_pairs = []
# scores = []
# preds = []
# gt = []
# for data in tqdm(test_dataset):
#     phrases = data['phrases']
#     phrase_pairs.append(phrases)

#     score = text_cos_sim(phrases)
#     scores.append(score)

#     preds.append(score>thres)
#     gt.append(data['label'])

In [10]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.metrics import confusion_matrix

# # Replace actual_labels and predicted_labels with your own data
# cm = confusion_matrix(gt, preds)

# # Define class labels
# class_labels = ['Negative', 'Positive']

# # Create a heatmap of the confusion matrix
# plt.figure(figsize=(8, 6))
# sns.set(font_scale=1.2)  # Adjust font size
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels, cbar=False)

# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.title('Confusion Matrix')
# plt.show()

# from sklearn.metrics import precision_score, recall_score, f1_score

# # Calculate precision
# precision = precision_score(gt, preds)

# # Calculate recall
# recall = recall_score(gt, preds)

# # Calculate F1 score
# f1 = f1_score(gt, preds)

# # Calculate accuracy
# acc = sum(x==y for x,y in zip(gt,preds))/len(gt)

# print(f'Accuracy: {acc}')
# print(f'Precision: {precision}')
# print(f'Recall: {recall}')
# print(f'F1 Score: {f1}')

In [13]:
text_ft[0].shape

torch.Size([768])

In [14]:
demo_phrases = test_dataset[1]['phrases']
demo_phrases

['Several climbers', 'a rock face']

In [15]:
demo_phrase = demo_phrases[1]
phr_tkn = clip.tokenize(demo_phrase).to(GPU)
phr_ft = model.encode_text(phr_tkn)
phr_ft

tensor([[-5.3223e-01, -6.5771e-01, -8.0078e-02, -1.8982e-01, -2.0654e-01,
          1.2177e-01,  3.6108e-01,  2.6489e-01, -2.2693e-01, -5.4092e-03,
          1.0187e-01, -6.9922e-01, -5.8984e-01,  4.3384e-01, -5.2100e-01,
         -2.6538e-01,  9.9170e-01,  3.7598e-01,  3.3228e-01, -3.9258e-01,
         -1.8970e-01,  2.4890e-01,  1.5710e-01, -1.2427e-01, -3.1396e-01,
         -4.3823e-01,  1.0146e+00, -2.2913e-01,  5.5756e-02,  1.8262e-01,
         -6.7139e-01, -2.5342e-01,  4.1455e-01,  5.0977e-01,  1.7346e-01,
         -2.0117e-01, -2.8870e-02, -1.2354e-01, -7.3975e-02, -2.6807e-01,
         -6.3591e-03,  1.5613e-01,  4.0625e-01, -4.5532e-01, -1.2695e-01,
         -6.0364e-02,  3.7939e-01, -2.0752e-01,  5.1855e-01, -3.8208e-01,
          2.5513e-01,  3.2379e-02,  2.3682e-01, -3.3984e-01,  2.4683e-01,
         -1.5625e-01,  1.7664e-01,  3.8013e-01, -7.4463e-02, -1.2427e-01,
          5.6445e-01,  7.2803e-01, -6.9092e-01,  1.3782e-01, -1.1709e+00,
          7.4463e-01,  3.2764e-01,  1.

In [17]:
phr_ft[0].shape

torch.Size([768])

In [18]:
import numpy as np

emb = np.load('/work/adapting-CLIP-VGPs/data/flickr/phrase_embs/wet hair.npy')

In [20]:
emb.shape

(1, 768)

In [27]:
! du -h /work/adapting-CLIP-VGPs/data/flickr/phrase_embs

29M	/work/adapting-CLIP-VGPs/data/flickr/phrase_embs


In [29]:
import torch
emb_tensor = torch.from_numpy(emb)
emb_tensor.shape

torch.Size([1, 768])